# PPK processing of "Stop and Go" data

**Script prepared by A. Rovere - MARUM, University of Bremen**

This script can be used to process GNSS data acquired in "stop-and-go" mode. The script needs the following inputs:

 - Rover data processed with RTKlib as kinematic points and saved as *.pos* file
 - Files exported from the data collector in *.csv* format

The script first merges the data collector files into a single dataframe. Then, postprocessed rover data is imported and a new dataframe is created with time-averaged postprocessed static positions acquired in FIX status. Time-averaged positions are also calculated for FLOAT and SINGLE status datapoints. All the results are saved in a multi-sheet excel file.

For a guide on how to use the NRCAN system and RTKlib with EMLID GPS, see:

https://docs.emlid.com/reach/common/tutorials/ppp-introduction/

This discussion on the EMLID forum contains some useful insights on the processing, as well another (similar) tool, including an intuitive user interface. The results of this script compare well with those obtained from this tool.

https://community.emlid.com/t/ppk-point-extractor-software/12822/46

## RTKlib setup
In RTKlib, remember to set up **output time in UTC**

For a guide on how to process your data with RTKlib, see:

https://docs.emlid.com/reach/common/tutorials/gps-post-processing/

A typical RTKlib configuration is annexed in the script main folder.

In [2]:
#Packages needed
import pandas as pd
import glob
import os
from pathlib import Path
import PyPDF2
import sys
import xlsxwriter as writer
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import ipywidgets as widgets


## Input folders and data
Indicate the folders where data is contained and the output file name

In [3]:
#Output file name
out='Example_output.xlsx'
#Folder where the csv file from the data collector is stored
csv_folder = r'\Example_data\Rover\Data_collector'
#Folder where pos files from RTKlib processing are stored
processed_data = r'\Example_data\Rover\Processed'

Indicate the 2-sigma uncertainties for Latitude, Longitude and Ellipsoid Height of the base station. These values will be taken into account when calculating final point uncertainties.

In [4]:
#95% sigma Latitude
LATsigma = 0.102
#95% sigma Longitude
LONsigma = 0.193
#95% sigma Ellipsoid height
Hsigma = 0.198
#Number of rows to skip in the pos file
num=9

# Import data collector files
Include in one folder all the *.csv* files exported from the data collector (Apple or Android) with ReachView software.

In [5]:
dirname = os.path.realpath('')+csv_folder
all_files = glob.glob(dirname + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['filename'] = os.path.basename(filename)
    li.append(df)
rawpoints = pd.concat(li, axis=0, ignore_index=True)

In [6]:
style = {'description_width': 'initial'}
name=widgets.Dropdown(
    options=rawpoints.columns,
    description='Name of point',
    disabled=False,layout={'width': 'max-content'},style=style)
start=widgets.Dropdown(
    options=rawpoints.columns,
    description='Start of data collection (yyyy-mm-dd UTC)',
    disabled=False,layout={'width': 'max-content'},style=style)
end=widgets.Dropdown(
    options=rawpoints.columns,
    description='End of data collection (yyyy-mm-dd UTC)',
    disabled=False,layout={'width': 'max-content'},style=style)
count=widgets.Dropdown(
    options=rawpoints.columns,
    description='Total points sampled in the field',
    disabled=False,layout={'width': 'max-content'},style=style)
ant_h=widgets.Dropdown(
    options=rawpoints.columns,
    description='Rover Antenna height (m)',
    disabled=False,
    layout={'width': 'max-content'},style=style)

display(name, start, end, count,ant_h)

Dropdown(description='Name of point', layout=Layout(width='max-content'), options=('index', 'name', 'longitude…

Dropdown(description='Start of data collection (yyyy-mm-dd UTC)', layout=Layout(width='max-content'), options=…

Dropdown(description='End of data collection (yyyy-mm-dd UTC)', layout=Layout(width='max-content'), options=('…

Dropdown(description='Total points sampled in the field', layout=Layout(width='max-content'), options=('index'…

Dropdown(description='Rover Antenna height (m)', layout=Layout(width='max-content'), options=('index', 'name',…

In [7]:
rawpoints[start.value] = rawpoints[start.value].astype('datetime64[ns]')
rawpoints[end.value] = rawpoints[end.value].astype('datetime64[ns]')
rawpoints.sort_values(by=[start.value],inplace=True)
rawpoints.reset_index(inplace=True)
rawpoints.rename(columns={name.value:'POINT ID',
                          start.value:'Start of data collection (yyyy-mm-dd UTC)',
                         end.value:'End of data collection (yyyy-mm-dd UTC)',
                         count.value:'Total points sampled in the field',
                         ant_h.value:'Rover Antenna height (m)'},inplace=True)
rawpoints.drop(columns=['level_0','index'],inplace=True)


writer = pd.ExcelWriter(out, engine='xlsxwriter',
                        options={'strings_to_numbers': True,
                                 'strings_to_formulas': False})
workbook=writer.book
wrap = workbook.add_format({'text_wrap': True, 
                            'valign':'vcenter',
                            'align':'center'})

rawpoints.to_excel(writer, sheet_name='Raw collector data', index=False)
worksheet = writer.sheets['Raw collector data']
worksheet.set_column('A:ZZ',20,wrap)
header_format = workbook.add_format({'bold': True,'text_wrap': True,'valign': 'vcenter','align':'center',
                                     'fg_color':'#C0C0C0','border': 1})
for col_num, value in enumerate(rawpoints.columns.values):
    worksheet.write(0, col_num, value, header_format)

# Postprocessed GPS points
Include in one folder all the *.pos* data processed with RTKlib. The data need to be processed as "kinematic" points. For each point included in the data collector files, the script selects the corresponding processed points following the start and end timestamps. The script then calculates average and ±1sigma positions based on the postprocessed rover kinematic data. A further calculation of uncertainties incorporates the base station sigma values in the final uncertainties.

In [8]:
dirname = os.path.realpath('')+processed_data
all_files = glob.glob(dirname + "/*.pos")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0,skiprows=num,delim_whitespace=True,parse_dates=[['%', 'UTC']])
    li.append(df)
processed = pd.concat(li, axis=0, ignore_index=True)
processed.reset_index(inplace=True)
processed['%_UTC'] = pd.to_datetime(processed['%_UTC'])

# Process points with a FIX solution available
processedQ1=processed.loc[processed['Q'] == 1]
dataQ1 = []
for index, row in rawpoints.iterrows():
    time_start=rawpoints['Start of data collection (yyyy-mm-dd UTC)'].values[index]
    time_end=rawpoints['End of data collection (yyyy-mm-dd UTC)'].values[index]
    processed_clip=processedQ1[(processedQ1['%_UTC'] >= time_start) & (processedQ1['%_UTC'] <=time_end)]
    row['Average postprocessed antenna Ellipsoid height (m)']=processed_clip['height(m)'].mean()
    row['Postprocessed Ellipsoid height ±2sigma (m)']=processed_clip['height(m)'].std()*2
    row['Average postprocessed latitude (degrees)']=processed_clip['latitude(deg)'].mean()
    row['Postprocessed latitude ±2sigma (degrees)']=processed_clip['latitude(deg)'].std()*2
    row['Average postprocessed longitude (degrees)']=processed_clip['longitude(deg)'].mean()
    row['Postprocessed Longitude ±2sigma (degrees)']=processed_clip['longitude(deg)'].std()*2
    row['Number of fix points processed']=processed_clip.loc[processed_clip.Q == 1, 'Q'].count()
    dataQ1.append(row)
dataQ1 = pd.DataFrame(dataQ1)
dataQ1 = dataQ1.dropna()
dataQ1['Ellipsoid heigh corrected for Rover antenna height (m)']=dataQ1['Average postprocessed antenna Ellipsoid height (m)']-dataQ1['Rover Antenna height (m)']
dataQ1['Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ1['Postprocessed Ellipsoid height ±2sigma (m)'])+np.square(float(Hsigma)))
dataQ1['Postprocessed Latitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ1['Postprocessed latitude ±2sigma (degrees)'])+np.square(float(LATsigma)))
dataQ1['Postprocessed Longitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ1['Postprocessed Longitude ±2sigma (degrees)'])+np.square(float(LONsigma)))

dataQ1 = dataQ1[['filename','POINT ID','Start of data collection (yyyy-mm-dd UTC)','End of data collection (yyyy-mm-dd UTC)',
                 'Total points sampled in the field','Number of fix points processed','Rover Antenna height (m)',
                 'Average postprocessed antenna Ellipsoid height (m)',
                 'Ellipsoid heigh corrected for Rover antenna height (m)','Postprocessed Ellipsoid height ±2sigma (m)',
                 'Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)',
                 'Average postprocessed latitude (degrees)','Postprocessed latitude ±2sigma (degrees)',
                 'Postprocessed Latitude including base uncertainty ±2sigma (m)',
                 'Average postprocessed longitude (degrees)','Postprocessed Longitude ±2sigma (degrees)',
                 'Postprocessed Longitude including base uncertainty ±2sigma (m)']]
#Write Q1 excel
if not dataQ1.empty:
    dataQ1.to_excel(writer, sheet_name='Postprocessed GPS points FIX', index=False)
    worksheet = writer.sheets['Postprocessed GPS points FIX']
    worksheet.set_column('A:ZZ',20,wrap)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,'valign': 'vcenter','align':'center',
                                     'fg_color':'#C0C0C0','border': 1})
    for col_num, value in enumerate(dataQ1.columns.values):
        worksheet.write(0, col_num, value, header_format)

# Process points with a FLOAT solution available
processedQ2=processed.loc[processed['Q'] == 2]
dataQ2 = []
for index, row in rawpoints.iterrows():
    time_start=rawpoints['Start of data collection (yyyy-mm-dd UTC)'].values[index]
    time_end=rawpoints['End of data collection (yyyy-mm-dd UTC)'].values[index]
    processed_clip=processedQ2[(processedQ2['%_UTC'] >= time_start) & (processedQ2['%_UTC'] <=time_end)]
    row['Average postprocessed antenna Ellipsoid height (m)']=processed_clip['height(m)'].mean()
    row['Postprocessed Ellipsoid height ±2sigma (m)']=processed_clip['height(m)'].std()*2
    row['Average postprocessed latitude (degrees)']=processed_clip['latitude(deg)'].mean()
    row['Postprocessed latitude ±2sigma (degrees)']=processed_clip['latitude(deg)'].std()*2
    row['Average postprocessed longitude (degrees)']=processed_clip['longitude(deg)'].mean()
    row['Postprocessed Longitude ±2sigma (degrees)']=processed_clip['longitude(deg)'].std()*2
    row['Number of float points processed']=processed_clip.loc[processed_clip.Q == 2, 'Q'].count()
    dataQ2.append(row)
dataQ2 = pd.DataFrame(dataQ2)
dataQ2 = dataQ2.dropna()
dataQ2['Ellipsoid heigh corrected for Rover antenna height (m)']=dataQ2['Average postprocessed antenna Ellipsoid height (m)']-dataQ2['Rover Antenna height (m)']
dataQ2['Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ2['Postprocessed Ellipsoid height ±2sigma (m)'])+np.square(float(Hsigma)))
dataQ2['Postprocessed Latitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ2['Postprocessed latitude ±2sigma (degrees)'])+np.square(float(LATsigma)))
dataQ2['Postprocessed Longitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ2['Postprocessed Longitude ±2sigma (degrees)'])+np.square(float(LONsigma)))

dataQ2 = dataQ2[['filename','POINT ID','Start of data collection (yyyy-mm-dd UTC)','End of data collection (yyyy-mm-dd UTC)',
                 'Total points sampled in the field','Number of float points processed','Rover Antenna height (m)',
                 'Average postprocessed antenna Ellipsoid height (m)',
                 'Ellipsoid heigh corrected for Rover antenna height (m)','Postprocessed Ellipsoid height ±2sigma (m)',
                 'Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)',
                 'Average postprocessed latitude (degrees)','Postprocessed latitude ±2sigma (degrees)',
                 'Postprocessed Latitude including base uncertainty ±2sigma (m)',
                 'Average postprocessed longitude (degrees)','Postprocessed Longitude ±2sigma (degrees)',
                 'Postprocessed Longitude including base uncertainty ±2sigma (m)']]
#Write Q2 excel
if not dataQ2.empty:
    dataQ2.to_excel(writer, sheet_name='Postprocessed GPS points FLOAT', index=False)
    worksheet = writer.sheets['Postprocessed GPS points FLOAT']
    worksheet.set_column('A:ZZ',20,wrap)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                         'valign': 'vcenter','align':'center',
                                         'fg_color':'#C0C0C0','border': 1})
    for col_num, value in enumerate(dataQ2.columns.values):
        worksheet.write(0, col_num, value, header_format)

# Process points with a SBAS solution available
processedQ3=processed.loc[processed['Q'] == 3]
dataQ3 = []
for index, row in rawpoints.iterrows():
    time_start=rawpoints['Start of data collection (yyyy-mm-dd UTC)'].values[index]
    time_end=rawpoints['End of data collection (yyyy-mm-dd UTC)'].values[index]
    processed_clip=processedQ3[(processedQ3['%_UTC'] >= time_start) & (processedQ3['%_UTC'] <=time_end)]
    row['Average postprocessed antenna Ellipsoid height (m)']=processed_clip['height(m)'].mean()
    row['Postprocessed Ellipsoid height ±2sigma (m)']=processed_clip['height(m)'].std()*2
    row['Average postprocessed latitude (degrees)']=processed_clip['latitude(deg)'].mean()
    row['Postprocessed latitude ±2sigma (degrees)']=processed_clip['latitude(deg)'].std()*2
    row['Average postprocessed longitude (degrees)']=processed_clip['longitude(deg)'].mean()
    row['Postprocessed Longitude ±2sigma (degrees)']=processed_clip['longitude(deg)'].std()*2
    row['Number of sbas points processed']=processed_clip.loc[processed_clip.Q == 3, 'Q'].count()
    dataQ3.append(row)
dataQ3 = pd.DataFrame(dataQ3)
dataQ3 = dataQ3.dropna()
dataQ3['Ellipsoid heigh corrected for Rover antenna height (m)']=dataQ3['Average postprocessed antenna Ellipsoid height (m)']-dataQ3['Rover Antenna height (m)']
dataQ3['Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ3['Postprocessed Ellipsoid height ±2sigma (m)'])+np.square(float(Hsigma)))
dataQ3['Postprocessed Latitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ3['Postprocessed latitude ±2sigma (degrees)'])+np.square(float(LATsigma)))
dataQ3['Postprocessed Longitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ3['Postprocessed Longitude ±2sigma (degrees)'])+np.square(float(LONsigma)))

dataQ3 = dataQ3[['filename','POINT ID','Start of data collection (yyyy-mm-dd UTC)','End of data collection (yyyy-mm-dd UTC)',
                 'Total points sampled in the field','Number of sbas points processed','Rover Antenna height (m)',
                 'Average postprocessed antenna Ellipsoid height (m)',
                 'Ellipsoid heigh corrected for Rover antenna height (m)','Postprocessed Ellipsoid height ±2sigma (m)',
                 'Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)',
                 'Average postprocessed latitude (degrees)','Postprocessed latitude ±2sigma (degrees)',
                 'Postprocessed Latitude including base uncertainty ±2sigma (m)',
                 'Average postprocessed longitude (degrees)','Postprocessed Longitude ±2sigma (degrees)',
                 'Postprocessed Longitude including base uncertainty ±2sigma (m)']]
#Write Q3 excel
if not dataQ3.empty:
    dataQ3.to_excel(writer, sheet_name='Postprocessed GPS points SBAS', index=False)
    worksheet = writer.sheets['Postprocessed GPS points SBAS']
    worksheet.set_column('A:ZZ',20,wrap)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                         'valign': 'vcenter','align':'center',
                                         'fg_color':'#C0C0C0','border': 1})
    for col_num, value in enumerate(dataQ3.columns.values):
        worksheet.write(0, col_num, value, header_format)

# Process points with a SINGLE solution available
processedQ5=processed.loc[processed['Q'] == 5]
dataQ5 = []
for index, row in rawpoints.iterrows():
    time_start=rawpoints['Start of data collection (yyyy-mm-dd UTC)'].values[index]
    time_end=rawpoints['End of data collection (yyyy-mm-dd UTC)'].values[index]
    processed_clip=processedQ5[(processedQ5['%_UTC'] >= time_start) & (processedQ5['%_UTC'] <=time_end)]
    row['Average postprocessed antenna Ellipsoid height (m)']=processed_clip['height(m)'].mean()
    row['Postprocessed Ellipsoid height ±2sigma (m)']=processed_clip['height(m)'].std()*2
    row['Average postprocessed latitude (degrees)']=processed_clip['latitude(deg)'].mean()
    row['Postprocessed latitude ±2sigma (degrees)']=processed_clip['latitude(deg)'].std()*2
    row['Average postprocessed longitude (degrees)']=processed_clip['longitude(deg)'].mean()
    row['Postprocessed Longitude ±2sigma (degrees)']=processed_clip['longitude(deg)'].std()*2
    row['Number of single points processed']=processed_clip.loc[processed_clip.Q == 5, 'Q'].count()
    dataQ5.append(row)
dataQ5 = pd.DataFrame(dataQ5)
dataQ5 = dataQ5.dropna()
dataQ5['Ellipsoid heigh corrected for Rover antenna height (m)']=dataQ5['Average postprocessed antenna Ellipsoid height (m)']-dataQ5['Rover Antenna height (m)']
dataQ5['Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ5['Postprocessed Ellipsoid height ±2sigma (m)'])+np.square(float(Hsigma)/2))
dataQ5['Postprocessed Latitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ5['Postprocessed latitude ±2sigma (degrees)'])+np.square(float(LATsigma)/2))
dataQ5['Postprocessed Longitude including base uncertainty ±2sigma (m)']=np.sqrt(np.square(dataQ5['Postprocessed Longitude ±2sigma (degrees)'])+np.square(float(LONsigma)/2))

dataQ5 = dataQ5[['filename','POINT ID','Start of data collection (yyyy-mm-dd UTC)','End of data collection (yyyy-mm-dd UTC)',
                 'Total points sampled in the field','Number of single points processed','Rover Antenna height (m)',
                 'Average postprocessed antenna Ellipsoid height (m)',
                 'Ellipsoid heigh corrected for Rover antenna height (m)','Postprocessed Ellipsoid height ±2sigma (m)',
                 'Postprocessed Ellipsoid height including base uncertainty ±2sigma (m)',
                 'Average postprocessed latitude (degrees)','Postprocessed latitude ±2sigma (degrees)',
                 'Postprocessed Latitude including base uncertainty ±2sigma (m)',
                 'Average postprocessed longitude (degrees)','Postprocessed Longitude ±2sigma (degrees)',
                 'Postprocessed Longitude including base uncertainty ±2sigma (m)']]
#Write Q5 excel
if not dataQ5.empty:
    dataQ5.to_excel(writer, sheet_name='Postprocessed GPS points SINGLE', index=False)
    worksheet = writer.sheets['Postprocessed GPS points SINGLE']
    worksheet.set_column('A:ZZ',20,wrap)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                         'valign': 'vcenter','align':'center',
                                         'fg_color':'#C0C0C0','border': 1})
    for col_num, value in enumerate(dataQ5.columns.values):
        worksheet.write(0, col_num, value, header_format)        
        
workbook.close()

## Description of outputs
The output of this script is an excel file (*.xslx*) containing different sheets.
 - **Raw collector data.** These are the data downloaded from the collector, with minimal adustments on column names
 - **Postprocessed GPS points FIX/FLOAT/SBAS/SINGLE.** Postprocessed FIX, FLOAT SBAS or SINGLE points organized in different sheets.
 
#### Legend for Postprocessed GPS points

***filename:*** The original data collector *csv* file.

***POINT ID:*** Original point ID as assigned during data collection. 

***Start of data collection (yyyy-mm-dd UTC):*** Start of raw data collection.

***End of data collection (yyyy-mm-dd UTC):*** End of raw data collection.

***Total points sampled in the field:*** Total points sampled during the survey in static mode.

***Number of fix points processed:*** Total points processed, concurring to the average calculations.

***Rover Antenna height (m):*** Rover antenna height as assigned originally in the field.

***Average postprocessed antenna Ellipsoid height (m):*** Average of postprocessed ellipsoid heights.

***Ellipsoid heigh corrected for Rover antenna height (m):*** Average of postprocessed ellipsoid heights corrected for antenna height. **Use this value as final elevation.** 

***Postprocessed Ellipsoid height 2sigma (m):*** Standard deviation of postprocessed ellipsoid heights.

***Postprocessed Ellipsoid height including base uncertainty 2sigma (m):*** Standard deviation of postprocessed ellipsoid heights, including uncertainty propagated in root mean square from the base processing. **Use this value as final elevation uncertainty.**

***Average postprocessed latitude (degrees):*** Average of postprocessed latitude corrected for antenna height. **Use this value as final latitude.** 

***Postprocessed latitude 2sigma (degrees):*** Standard deviation of postprocessed latitude values.

***Postprocessed Latitude including base uncertainty 2sigma (m):*** Standard deviation of postprocessed latitude values, including uncertainty propagated in root mean square from the base processing. **Use this value as final latitude uncertainty.**

***Average postprocessed longitude (degrees):*** Average of postprocessed longitude corrected for antenna height. **Use this value as final longitude.** 

***Postprocessed longitude 2sigma (degrees):*** Standard deviation of postprocessed longitude values.

***Postprocessed longitude including base uncertainty 2sigma (m):*** Standard deviation of postprocessed longitude values, including uncertainty propagated in root mean square from the base processing. **Use this value as final longitude uncertainty.**

***
## License
This software is relased under the MIT license.

Copyright 2020 Alessio Rovere

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
***
# Research funding acknowledgments
This script and associated data were created in the framework of the European Reasearch Council Starting Grant WARMCOASTS (Grant Agreement Number 802414), funded under the European Union's Horizon 2020 research and Innovation programme.
***
# Code acknowledgments
https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f
https://kite.com/python/answers/how-to-redirect-print-output-to-a-text-file-in-python
https://stackoverflow.com/questions/41857659/python-pandas-add-filename-column-csv
https://stackoverflow.com/questions/46184239/extract-a-page-from-a-pdf-as-a-jpeg